In [20]:
# MySQL 데이터베이스 연결 설정
import pymysql

connection = pymysql.connect(
    host="localhost",
    user="hwooks",
    password="hwooks",
    db="hwooks_db",
)

cursor = connection.cursor()

In [12]:
# 테이블 생성
create_table_query = """
CREATE TABLE IF NOT EXISTS customer (
    id INT AUTO_INCREMENT PRIMARY KEY,
    username VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    age INT
)
"""
cursor.execute(create_table_query)
connection.commit()

In [13]:
# 컬럼 추가
# 이 컬럼은 timestamp 데이터 타입, 기본값으로 현재 타임스탬프(current_timestamp)가 설정. 즉 새로운 레코드가 삽입될때 자동으로 현재 시간이 기록
alter_table_query = "ALTER TABLE customer ADD COLUMN created_at timestamp DEFAULT CURRENT_TIMESTAMP"
cursor.execute(alter_table_query)
connection.commit()

In [14]:
# email 컬럼에 NOT NULL 제약조건을 삭제
alter_table_query = "ALTER TABLE customer MODIFY email VARCHAR(100)"
cursor.execute(alter_table_query)
connection.commit()

In [11]:
## 테이블 삭제
drop_table_query="DROP TABLE IF EXISTS customer"
cursor.execute(drop_table_query)
connection.commit()

In [15]:
## 데이터 입력
# %s 는 mysql.connector 모듈에서 사용되는 플레이스홀더입니다. 이 플레이스홀더는 데이터를 삽입할때 사용. (sqlight ?)
insert_query="INSERT INTO customer (username, email, age) VALUES (%s, %s, %s)"
data=("john_doe", "john@example.com", 30)
cursor.execute(insert_query, data)
connection.commit()

In [17]:
## 데이터 조회
select_query="SELECT * FROM customer"
cursor.execute(select_query)
results=cursor.fetchall()
for row in results:
    print(row)
    

(1, 'john_doe', 'john@example.com', 31, datetime.datetime(2024, 7, 15, 12, 11, 30))


In [21]:
## 데이터 수정
update_query="UPDATE customer SET age= %s WHERE username=%s"
cursor.execute(update_query, (31,"john_doe"))
connection.commit()

In [22]:
## 데이터 삭제
delete_query="DELETE FROM customer WHERE username = %s"
cursor.execute(delete_query, ("john_doe"))
connection.commit()

In [32]:
## 데이터 반복 삽입
customers=[
    ("alice", 'alice@exmple.com', 25),
    ("bob", 'bob@exmple.com', 28),
    ("charlie", 'charlie@exmple.com', 22)
]
cursor.executemany(insert_query, customers)
connection.commit()

IntegrityError: (1062, "Duplicate entry 'alice@exmple.com' for key 'buyer.email'")

In [33]:
## 데이터 조회
select_query="SELECT * FROM customer"
cursor.execute(select_query)
results=cursor.fetchall()
for row in results:
    print(row)
    

(2, 'alice', 'alice@exmple.com', 25, datetime.datetime(2024, 7, 15, 12, 22, 8))
(3, 'bob', 'bob@exmple.com', 28, datetime.datetime(2024, 7, 15, 12, 22, 8))
(4, 'charlie', 'charlie@exmple.com', 22, datetime.datetime(2024, 7, 15, 12, 22, 8))


In [34]:
# 데이터 조회 결과를 dataframe으로 변환
import pandas as pd
# 컬럼 이름 가져오기
column_names=[desc[0] for desc in cursor.description]

# dataframe으로 변환
df=pd.DataFrame(results,columns=column_names)

# dataframe 출력

print(df)

   id username               email  age          created_at
0   2    alice    alice@exmple.com   25 2024-07-15 12:22:08
1   3      bob      bob@exmple.com   28 2024-07-15 12:22:08
2   4  charlie  charlie@exmple.com   22 2024-07-15 12:22:08


In [35]:
df.to_csv("customers.csv", index=False)


In [36]:
# buyer 테이블 생성
create_table_query = """
    CREATE TABLE IF NOT EXISTS buyer (
        id INT AUTO_INCREMENT PRIMARY KEY,
        username VARCHAR(50) NOT NULL,
        email VARCHAR(100) UNIQUE NOT NULL,
        age INT,
        created_at timestamp DEFAULT CURRENT_TIMESTAMP
    )
"""
cursor.execute(create_table_query)
connection.commit()

In [ ]:
# csv 파일 읽어서 데이터베이스에 삽입
import csv

# CSV 읽기
file_path='customers.csv'
df=pd.read_csv(file_path)

# 필요한 컬럼만 선택
data_to_insert=df[['username', 'email', 'age']].values.tolist()

#SQL 쿼리 정의
insert_query="INSERT INTO buyer (username, email, age) VALUES (%s, %s, %s)"

# 데이터베이스에 삽입
cursor.executemany(insert_query, data_to_insert)
connection.commit()



In [38]:
# 데이터 조회 및 dataframe 변환
select_query="SELECT * FROM buyer"
cursor.execute(select_query)
results=cursor.fetchall()

# 컬럼 이름 가져오기
column_names=[desc[0] for desc in cursor.description]

# dataframe으로 변환
df_results=pd.DataFrame(results, columns=column_names)

# dataframe 출력
print(df_results)

# 연결 종료

cursor.close()
connection.close()

   id username               email  age          created_at
0   1    alice    alice@exmple.com   25 2024-07-15 12:38:42
1   2      bob      bob@exmple.com   28 2024-07-15 12:38:42
2   3  charlie  charlie@exmple.com   22 2024-07-15 12:38:42


In [18]:
#연결 종료
cursor.close()
connection.close()